# Creating Clusters for Stochastic Optimization
---

Import packages

In [191]:
from pathlib import Path
import pickle
import pandas as pd
import geopandas as gpd
import json
import numpy as np

Import cleaned traces

In [176]:
export_fp = Path.home() / 'Downloads/cleaned_trips'
#network_fp = Path.home() / "Downloads/cleaned_trips/networks/final_network.gpkg"

#load all traces
with (export_fp/'cleaned_traces.pkl').open('rb') as fh:
    coords_dict, trips_df = pickle.load(fh)

In [177]:
trips_df.loc[trips_df['tripid']==80]

,tripid,initial_start_time,initial_end_time,initial_duration,initial_total_points,initial_avg_accuracy,tot_points,duration,min_time_difference,max_time_difference,mean_time_difference,min_distance_ft,max_distance_ft,avg_distance_ft,total_distance_ft,max_speed_mph,min_speed_mph,avg_speed_mph,total_points
12,80,2012-10-12 11:38:29,2012-10-12 11:48:00,0 days 00:09:31,569,53.368994,546.0,0 days 00:08:59,0 days 00:00:01,0 days 00:00:48,0 days 00:00:01.871527777,5.260514,330.793547,26.254532,7561.305097,31.185259,0.660025,14.300888,289.0


Trip data

In [178]:
trip = pd.read_csv(export_fp/"trip.csv", header = None)
col_names = ['tripid','userid','trip_type','description','starttime','endtime','num_points']
trip.columns = col_names
trip.drop(columns=['starttime','endtime','num_points'],inplace=True)

User data

In [179]:
user = pd.read_csv(export_fp/"user.csv", header=None)
user_col = ['userid','created_date','device','email','age',
            'gender','income','ethnicity','homeZIP','schoolZip',
            'workZip','cycling_freq','rider_history','rider_type','app_version']
user.columns = user_col
user.drop(columns=['device','app_version','app_version','email'],inplace=True)

Add trip and user data to trips_df

In [180]:
trips_df = pd.merge(trips_df,trip,on='tripid').merge(user,on='userid').copy()

In [181]:
fp = Path.home() / "Documents/GitHub/Impedance-Calibration"
user_data_definitions = json.load(open(fp/'user_data_definition.json'))
for col in trips_df.columns:
    if col in user_data_definitions.keys():
        trips_df[col] = trips_df[col].astype(str)
        trips_df[col] = trips_df[col].map(user_data_definitions[col])

In [182]:
user_data_definitions['cycling_freq']

{'-1': 'no data',
 '0': 'no data',
 '1': 'Less than once a month',
 '2': 'Several times per month',
 '3': 'Several times per week',
 '4': 'Daily'}

Find Euclidean distance between origin and destination

In [183]:
for tripid, coords in coords_dict.items():
    #get starting location
    start_lon = coords.loc[coords['datetime'].idxmin(),'lon'].item()
    start_lat = coords.loc[coords['datetime'].idxmin(),'lat'].item()

    #get ending location
    end_lon = coords.loc[coords['datetime'].idxmax(),'lon'].item()
    end_lat = coords.loc[coords['datetime'].idxmax(),'lat'].item()

    #assign to trips_df
    trips_df.at[trips_df['tripid']==tripid,'start_lon'] = start_lon
    trips_df.at[trips_df['tripid']==tripid,'start_lat'] = start_lat
    trips_df.at[trips_df['tripid']==tripid,'end_lon'] = end_lon
    trips_df.at[trips_df['tripid']==tripid,'end_lat'] = end_lat

# find euclidean distance between start and end coord (for finding loop trips)
start_geo = gpd.points_from_xy(trips_df['start_lon'],trips_df['start_lat'],crs='epsg:4326').to_crs('epsg:2240')
end_geo = gpd.points_from_xy(trips_df['end_lon'],trips_df['end_lat'],crs='epsg:4326').to_crs('epsg:2240')
trips_df['euclidean_distance'] = start_geo.distance(end_geo)

Find origin and destination zip code

Remove loops and exercies trips

In [184]:
tolerance_ft = 1000
self_loops = trips_df['euclidean_distance'] < tolerance_ft
exercise = trips_df['trip_type'] == 'Exercise'
trips_df = trips_df[~exercise & ~self_loops]
print(self_loops.sum(),'trips were self loops')
print(exercise.sum(),'trips were for exercise')

2136 trips were self loops
2794 trips were for exercise


What variables do we have for clustering?

In [185]:
trips_df.columns

Index(['tripid', 'initial_start_time', 'initial_end_time', 'initial_duration',
       'initial_total_points', 'initial_avg_accuracy', 'tot_points',
       'duration', 'min_time_difference', 'max_time_difference',
       'mean_time_difference', 'min_distance_ft', 'max_distance_ft',
       'avg_distance_ft', 'total_distance_ft', 'max_speed_mph',
       'min_speed_mph', 'avg_speed_mph', 'total_points', 'userid', 'trip_type',
       'description', 'created_date', 'age', 'gender', 'income', 'ethnicity',
       'homeZIP', 'schoolZip', 'workZip', 'cycling_freq', 'rider_history',
       'rider_type', 'start_lon', 'start_lat', 'end_lon', 'end_lat',
       'euclidean_distance'],
      dtype='object')

In [186]:
trips_df[trips_df['rider_type'].isna()]

,tripid,initial_start_time,initial_end_time,initial_duration,initial_total_points,initial_avg_accuracy,tot_points,duration,min_time_difference,max_time_difference,...,schoolZip,workZip,cycling_freq,rider_history,rider_type,start_lon,start_lat,end_lon,end_lat,euclidean_distance


In [190]:
relveant_columns = ['tripid','userid','duration','total_distance_ft','avg_speed_mph','trip_type',
                    'age', 'gender', 'income', 'ethnicity','homeZIP', 'schoolZip', 'workZip',
                    'cycling_freq', 'rider_history','rider_type'
                    ]
trips_df = trips_df[relveant_columns].copy()

Replace no data with np.nan

In [192]:
trips_df.replace('no data',np.NaN,inplace=True)

In [193]:
trips_df.isna().sum().sort_values(ascending=False)

description          14925
cycling_freq          7271
income                6543
age                   5948
gender                5827
ethnicity             4946
rider_history         4359
rider_type            4249
created_date             0
userid                   0
trip_type                0
avg_speed_mph            0
homeZIP                  0
schoolZip                0
workZip                  0
total_distance_ft        0
duration                 0
tripid                   0
dtype: int64

-  Continuous
    - Travel time (minutes)
    - Total distance traveled (feet)
    - Average speed (miles per hour)
    - Shortest distance (feet)




- Categorical
    - Interval
        - Age
        - Income
    - Ordinal
        - Cycling frequency
        - Rider history
        - Rider type
    - Nominal
        - Gender (M/F)
        - Ethnicity
        - Trip type (could simplify to utlitarian/leisure)

In [194]:
post = pd.read_csv(fp/'trips_df_postmatch.csv')
post = post[['tripid','shortest_length','overlap_length','detour_rate']].copy()
trips_df = trips_df.merge(post,on='tripid').copy()

In [198]:
trips_df.groupby('userid')

userid
10       59.866667
14        5.760000
16      455.500000
17       38.000000
18      -62.200000
           ...    
1723     -5.882353
1725    -30.000000
1727    356.000000
1733    -11.000000
1738     -4.000000
Name: detour_rate, Length: 830, dtype: float64